**Library Package**

In [162]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

**Load Dataset**

In [155]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
#df

In [163]:
missing_col = ['col_4', 'col_10', 'col_11', 'col_12', 'col_19', 'col_21']
#df[missing_col].describe()
#df[missing_col].mode()
# col_10: try mode
# col_11: try mean
# col_12: try mode
# col_19: try mode
# col_21: try 
#missing_value_col = df[motherfucker]
#missing_value_col

**Data Preprocessing**

**Step1: Missing Value**

In [164]:
print(df.isnull().sum())
#print(df_test.isnull().sum())

col_1              0
col_2              0
col_3              0
col_4          12817
col_5              0
col_6              0
col_7              0
col_8              0
col_9              0
col_10          6004
col_11          1799
col_12          1661
col_13             0
col_14             0
col_15             0
col_16             0
col_17             0
col_18             0
col_19          3836
col_20             0
col_21            17
CreditScore        0
dtype: int64


In [165]:
# Fill the mean and median to deal with the missing value 
# We should clean col_4, 10, 11, 12, 19, 20, 21
# Since col_20 is categorical value, we use median
# Others are numeric features, we use mean
# If testing is not well, we can try put most frequent to fill in missing value

col_mean_fill = ['col_4', 'col_10', 'col_11', 'col_12', 'col_19', 'col_21']

# For numeric avriables
for col in col_mean_fill:
    df[col].fillna(df[col].median(), inplace = True)
    df_test[col].fillna(df_test[col].median(), inplace = True)

# For categorical variable
df['col_20'].fillna(df['col_20'].median(), inplace = True)
df_test['col_20'].fillna(df_test['col_20'].median(), inplace = True)

#df

In [167]:
#print(df.isnull().sum())
print(df_test.isnull().sum())

col_1     0
col_2     0
col_3     0
col_4     0
col_5     0
col_6     0
col_7     0
col_8     0
col_9     0
col_10    0
col_11    0
col_12    0
col_13    0
col_14    0
col_15    0
col_16    0
col_17    0
col_18    0
col_19    0
col_20    0
col_21    0
dtype: int64


In [168]:
# Do one-hot encoding on the categorical data
col_to_encoded = ['col_2', 'col_13', 'col_17', 'col_20']
df_preprocessed = pd.get_dummies(df, columns = col_to_encoded)
df_test_preprocessed = pd.get_dummies(df_test, columns = col_to_encoded)

len(df_preprocessed.columns.tolist())

47

In [169]:
X = df_preprocessed.drop('CreditScore', axis = 1)
y = df_preprocessed['CreditScore']

# Oversampling
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)

#sme = SMOTEENN(random_state=42)
#X_resampled, y_resampled = sme.fit_resample(X, y)

In [170]:
# Get training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, random_state = 0)

print('training data size:', len(X_train))
print('testing data set:', len(X_test))

training data size: 112171
testing data set: 28043


**Step 3: Standardization**

In [171]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

**Select the fucking features**

In [173]:
# 创建SelectKBest对象，指定互信息作为评估指标
selector = SelectKBest(score_func=mutual_info_classif, k=15)  # 选择得分最高的10个特征

X_train_std_fs = selector.fit_transform(X_train_std, y_train)
X_test_std_fs = selector.transform(X_test_std)

**PCA/T-SNE**

In [37]:
#pca = PCA(n_components = 15, whiten = True)
#tsne = TSNE(n_components = 3, random_state = 42)

#pca.fit(X_train_std)
#X_train_std_pca = pca.transform(X_train_std)
#X_test_std_pca = pca.transform(X_test_std)

#tsne.fit(X_train_std)
#X_train_std_tsne = tsne.transform(X_train_std)
#X_test_std_tsne = tsne.transform(X_test_std)

**Random Forest**

In [174]:
# Create a Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=200, criterion = 'gini', random_state=42)
# Train
rfc.fit(X_train_std_fs, y_train)

# Predict
y_pred_rfc = rfc.predict(X_test_std_fs)

# Evaluation
print('RFC result:', y_pred_rfc)
print('Misclassified samples: %d' %(y_test != y_pred_rfc).sum())
print('Accuracy: %.4f' %accuracy_score(y_test, y_pred_rfc)) 

# Show the F1-score
f1_scores = {}
for class_label in set(y_pred_rfc):
    y_true_f1 = [1 if label == class_label else 0 for label in y_test]
    y_pred_f1 = [1 if label == class_label else 0 for label in y_pred_rfc]

    f1_scores[class_label] = f1_score(y_true_f1, y_pred_f1)

for class_label, f in f1_scores.items():
    print("F1-score for class {}:{:.4f}".format(class_label, f))

avg_f1_score = sum(f1_scores.values()) / len(f1_scores)
print("Average F1 score across all classes: {:.4f}".format(avg_f1_score))

RFC result: [0 2 2 ... 2 1 1]
Misclassified samples: 3871
Accuracy: 0.8620
F1-score for class 0:0.8086
F1-score for class 1:0.9010
F1-score for class 2:0.8722
Average F1 score across all classes: 0.8606


**Predict test.csv**

In [137]:
# Take the features that we selected before
test_data = df_test_preprocessed

# Standardization
sc = StandardScaler()
sc.fit(test_data)
test_data_std = sc.transform(test_data)

# Select Features
test_data_std_fs = selector.transform(test_data_std)

fin_y_pred = rfc.predict(test_data_std_fs)
fin_y_pred
fin_df = pd.DataFrame({'label': fin_y_pred})
fin_df.to_csv('rfc_output_0412.csv', index = True, index_label = 'Id')